In [ ]:
%pip install -r requirements.txt

In [1]:
import requests
import os

In [3]:
while(True):
    try:
        cd:int = input("Introduzca el codigo nacional")
        url = f"https://cima.aemps.es/cima/rest/presentacion/{cd}"
        try:
            respuesta = requests.get(url)
            
            if respuesta.status_code == 200:
                datos = respuesta.json()
                nombre_medicamento = datos.get('nombre')
                print(f"Nombre: {nombre_medicamento}")
                for doc in datos['docs']:
                    if doc['tipo'] == 1:
                        try:
                            url_ficha_tecnica = doc['url']
                        except Exception as e:
                            print(f"Ocurrió un error a la hora de guardar la url del PDF: {e}")
                    elif doc['tipo'] == 2:
                        try:
                            url_prospecto= doc['url']
                        except Exception as e:
                            print(f"Ocurrió un error a la hora de guardar la url del PDF: {e}")
            else:
                print(f"Error: No se encontró el medicamento (Status: {respuesta.status_code})")

        except Exception as e:
            print(f"Ocurrió un error en la conexión: {e}")
        print(url_ficha_tecnica)
        print(url_prospecto)
        break
    except Exception as i:
        print("pon un codigo nacional valido")



Nombre: IBUPROFENO (ARGININA)  CINFA 600 mg GRANULADO PARA SOLUCION ORAL EFG , 500 sobres
https://cima.aemps.es/cima/pdfs/ft/67939/FT_67939.pdf
https://cima.aemps.es/cima/pdfs/p/67939/P_67939.pdf


In [ ]:

BASE_DIR = os.getcwd()

CARPETA_DESTINO = os.path.join(BASE_DIR, "data")

if not os.path.exists(CARPETA_DESTINO):
    os.makedirs(CARPETA_DESTINO)
    print(f"Carpeta creada en: {CARPETA_DESTINO}")

print("Descargando archivos...")
respuesta = requests.get(url_ficha_tecnica)
respuesta2 = requests.get(url_prospecto)

if respuesta.status_code == 200 and respuesta2.status_code == 200:
    nombre_ficha_tecnica = os.path.join(CARPETA_DESTINO, f"Ficha_{nombre_medicamento}.pdf")
    nombre_prospecto = os.path.join(CARPETA_DESTINO, f"Prospecto_{nombre_medicamento}.pdf")
    
    with open(nombre_ficha_tecnica, "wb") as archivo:
        archivo.write(respuesta.content)
    print(f"¡Hecho! Guardado en: {nombre_ficha_tecnica}")
    
    with open(nombre_prospecto, "wb") as archivo:
        archivo.write(respuesta2.content)
    print(f"¡Hecho! Guardado en: {nombre_prospecto}")
else:
    print(f"Error al descargar. Status codes: {respuesta.status_code}, {respuesta2.status_code}")

Descargando archivos...
¡Hecho! Guardado en: d:\Universidad\Proyecto_Odiseia4good\data\Ficha_IBUPROFENO (ARGININA)  CINFA 600 mg GRANULADO PARA SOLUCION ORAL EFG , 500 sobres.pdf
¡Hecho! Guardado en: d:\Universidad\Proyecto_Odiseia4good\data\Prospecto_IBUPROFENO (ARGININA)  CINFA 600 mg GRANULADO PARA SOLUCION ORAL EFG , 500 sobres.pdf


In [ ]:
archivos = [nombre_ficha_tecnica, nombre_prospecto]


for archivo in archivos:
    if os.path.exists(archivo):
        os.remove(archivo)
        print(f"Archivo '{os.path.basename(archivo)}' eliminado correctamente de /pdfs.")
    else:
        print(f"El archivo '{os.path.basename(archivo)}' no existía en la carpeta.")

Archivo 'Ficha_IBUPROFENO (ARGININA)  CINFA 600 mg GRANULADO PARA SOLUCION ORAL EFG , 500 sobres.pdf' eliminado correctamente de /pdfs.
Archivo 'Prospecto_IBUPROFENO (ARGININA)  CINFA 600 mg GRANULADO PARA SOLUCION ORAL EFG , 500 sobres.pdf' eliminado correctamente de /pdfs.
